In [5]:
# блиотеки
import pandas as pd
import sys
import os

In [22]:
# куда выплюнуть финальный файл
output_dir = os.getcwd() + '/reports/'
# откуда забираем, сейчас текущая папка скрипта + название папки
input_dir = os.getcwd() + '/xls_files/'

# если папки выгрузки нет то она создается
if os.path.isdir(output_dir):
    print(f'{output_dir} already exists')
else:
    print(output_dir)
    os.mkdir(f'{output_dir}') 
    
# читаем названия файлов из папки
files = os.listdir(input_dir)
files.sort()

/home/sergey/drclinics/reports/telemed/test_task/123/KNOMAS/reports/ already exists


In [62]:
# создаем финальную таблицу
df_final = None

for file in files:
    # получаем расштрение файла
    extension = file.split('.')[-1]
    # если расширение это xls\xlsx то открываем файл, иначе пропускаем
    if extension == 'xls' or extension == 'xlsx':
        
        # цикл по всем файлам из папки
        df_temp: pd.DataFrame = pd.read_excel(input_dir + file)

        # цепляем ключ (36 вместо 38 потому что заголовок не считается + нумерация идет с 0)
        k = df_temp.loc[36, 'Значение']
        # получаем DataFrame значений по ключу LINE
        val_df_temp = df_temp[df_temp['Техническое имя'] == 'LINE'].copy()
        
        val = list()
        # проходим по всем строкам временной таблицы с ключом LINE
        for val_temp in list(val_df_temp['Значение'].unique()):
            # делим значение по одинарным кавычкам на список
            for i, name in enumerate(val_temp.split("'")):
                # каждое нечетное значение базаем в список 
                if i % 2 == 1:
                    val.append(name)
        val = ', '.join(val)

        # если финальная таблица пустая то она создается из k, val
        if df_final is None:
            df_final = pd.DataFrame(zip([k], [val], [file]),
                   columns = ['Key', 'Value(s)', 'file'])
        # иначе к ней конкатинируется временная таблица
        else:
            df_temp = pd.DataFrame(zip([k], [val], [file]),
                   columns = ['Key', 'Value(s)', 'file'])

            df_final = pd.concat([df_final, df_temp]).reset_index(drop=True)
      
    else:
        pass
# сохраняем финальную табличку в эксель
df_final.to_excel(output_dir + 'final.xlsx')